# Collaborative Filtering Based Recommender System

In [9]:
import pandas as pd
import numpy as np

In [10]:
books=pd.read_csv('Books.csv') #pandas is used to read csv files
users=pd.read_csv('Users.csv')
ratings=pd.read_csv('Ratings.csv')

C:\Users\parve\AppData\Local\Temp\ipykernel_28212\3804135747.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books=pd.read_csv('Books.csv') #pandas is used to read csv files


In [11]:
print(books.columns)  # Lists all column names


Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')


# Feature Engineering

In [12]:
#feature enginnering right now
books=books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [13]:
books.head(2)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada


In [26]:
books.rename(columns={'Book-Title': 'title','Book-Author': 'author','Year-Of-Publication': 'year', 'Publisher': 'publisher' },inplace=True)

In [27]:
books.head(3)

,ISBN,title,author,year,publisher
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial


In [18]:
books.shape #no. of rows in the books.csv

(271360, 5)

In [15]:
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [28]:
users.rename(columns={'User-ID': 'user','Location': 'location','Age': 'age'},inplace=True)

In [30]:
users.head(2)

,user,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0


In [18]:
ratings.head(2)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5


In [31]:
ratings.rename(columns={'User-ID': 'user','Book-Rating': 'rating'},inplace=True)

In [32]:
ratings.head(2)

,user,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0


In [33]:
#we can take a matrix whre columns are users and rows are books. but this can lead to skewed interpretations. so we only take users who have rated more than 200 books and books than have been rated more than 50 times.
x=ratings['user'].value_counts()>200 #gives number of books rated by each individual user


In [34]:
x[x].shape #number of users that we will actually consider now

(899,)

In [35]:
y=x[x].index
ratings=ratings[ratings['user'].isin(y)] 
#join the data frames
ratings_with_books=ratings.merge(books,on='ISBN')

In [37]:
number_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()
number_rating.rename(columns={'rating':'number of ratings'},inplace=True)
final_rating=ratings_with_books.merge(number_rating,on='title')

In [38]:
final_rating=final_rating[final_rating['number of ratings']>=50]
final_rating.drop_duplicates(['user','title'],inplace=True)
book_pivot=final_rating.pivot_table(columns='user',index='title',values='rating')
book_pivot.fillna(0,inplace=True)

# Model Training

In [39]:
from scipy.sparse import csr_matrix
book_sparse=csr_matrix(book_pivot)
from sklearn.neighbors import NearestNeighbors
model=NearestNeighbors(algorithm='brute')

In [40]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [41]:
distances, suggestions=model.kneighbors(book_pivot.iloc[237, :].values.reshape(1,-1),n_neighbors=6)
#distances and suggestions are both arrays
for i in range(len(suggestions)):
    print(book_pivot.index[suggestions[i]])

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [45]:
#function that takes book's name and then give recommendations
np.where(book_pivot.index=='Animal Farm')[0][0] #gives book id
def recommend_book (book_name):
    book_id=np.where(book_pivot.index==book_name)[0][0]
    distances, suggestions=model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1,-1),n_neighbors=6)
    for i in range(len(suggestions)):
            print(book_pivot.index[suggestions[i]])

In [46]:
recommend_book('Animal Farm')

Index(['Animal Farm', 'Exclusive', 'Jacob Have I Loved', 'Second Nature',
       'Pleading Guilty', 'No Safe Place'],
      dtype='object', name='title')
